In [2]:
import numpy as np
import numpy.random as rand
from scipy.constants import codata
import scipy.interpolate as si
from todloop.utils.pixels import PixelReader
from todloop.base import Routine, TODLoop
import cPickle
import os
import pprint
from todloop.utils.cuts import pixels_affected_in_event
from plot_glitches.routines import PlotGlitches
import moby2
from moby2.scripting import get_filebase
from random import *
import matplotlib.pyplot as plt
import matplotlib.patches as patches

%pylab inline

Populating the interactive namespace from numpy and matplotlib


/mnt/act3/users/jpm136/miniconda2/lib/python2.7/site-packages/moby2/util/noninteractive_plots.py:2: UserWarning: 
This call to matplotlib.use() has no effect because the backend has already
been chosen; matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

The backend was *originally* set to 'TkAgg' by the following code:
  File "/mnt/act3/users/jpm136/miniconda2/lib/python2.7/runpy.py", line 174, in _run_module_as_main
    "__main__", fname, loader, pkg_name)
  File "/mnt/act3/users/jpm136/miniconda2/lib/python2.7/runpy.py", line 72, in _run_code
    exec code in run_globals
  File "/mnt/act3/users/jpm136/miniconda2/lib/python2.7/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/mnt/act3/users/jpm136/miniconda2/lib/python2.7/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/mnt/act3/users/jpm136/miniconda2/lib/python2.

In [8]:
"""
#-------------------------------------------------------------------------#
# Change the filepath to the corresponding TOD you want to analyze below! #
#-------------------------------------------------------------------------#
"""
fb = get_filebase()
filepath = "/mnt/act6/actpol/data/season5/merlin/14942/1494264511.1494301328.ar6.zip"
tod_name = filepath[43:68]
array = filepath[65:68].upper()
print array
print tod_name
tod_filename = fb.filename_from_name(tod_name, single=True)# get file path
print tod_filename 
tod_data = moby2.scripting.get_tod({'filename': tod_filename, 'repair_pointing': True})
cuts = moby2.scripting.get_cuts({'depot':'/mnt/act3/users/lmaurin/depot','tag':'MR1_P' + str(array[0]) + str(array[2]) + '_2017'},tod=tod_data)



#fix optical sign
optical_signs = tod_data.info.array_data['optical_sign']
tod_data.data = tod_data.data*optical_signs[:, np.newaxis]

#calibrate
cal = moby2.scripting.get_calibration({'type': 'iv', 'source': 'data'}, tod=tod_data)
cal_mask, cal_val = cal.get_property('cal', det_uid=tod_data.det_uid)
tod_data.data *= cal_val[:,None]
    
print cuts

AR6
1494264511.1494301328.ar6
/mnt/act6/actpol/data/season5/merlin/14942/1494264511.1494301328.ar6.zip


IOError: [Errno 2] No such file or directory: '/mnt/act3/users/lmaurin/depot/TODCuts/MR1_PA6_2017/14942/1494264511.1494301328.ar6.cuts'

In [7]:
def timeseries(pixel_id, s_time, e_time, buffer=10):

    start_time = s_time - buffer
    end_time = e_time + buffer

    a1, a2 = self._pr.get_f1(pixel_id)
    b1, b2 = self._pr.get_f2(pixel_id)
    d1, d2 = tod_data.data[a1], tod_data.data[a2]
    d3, d4 = tod_data.data[b1], tod_data.data[b2]

    # try to remove the mean from start_time to end_time                                                                                                                     
    d1 -= np.mean(d1[start_time:end_time])
    d2 -= np.mean(d2[start_time:end_time])
    d3 -= np.mean(d3[start_time:end_time])
    d4 -= np.mean(d4[start_time:end_time])

    time = tod_data.ctime - tod_data.ctime[0]
    time = time[start_time:end_time]

    d_1 = d1[start_time:end_time]
    d_2 = d2[start_time:end_time]
    d_3 = d3[start_time:end_time]
    d_4 = d4[start_time:end_time]


    return time, d_1, d_2, d_3, d_4

In [11]:
print cuts

{'coincident_signals': {'214': CutsVector([[25378, 25408]], dtype=int32), '212': CutsVector([[ 25384,  25389],
            [141706, 141709]], dtype=int32), '213': CutsVector([[ 25378,  25406],
            [ 63469,  63471],
            [101672, 101675],
            [127472, 127474],
            [187609, 187610],
            [192573, 192579],
            [209657, 209662]], dtype=int32), '669': CutsVector([], shape=(0, 2), dtype=int32), '668': CutsVector([[  3352,   3436],
            [  5085,   5094],
            [ 11685,  11772],
            [ 22445,  22453],
            [ 23307,  23310],
            [ 24818,  25913],
            [ 25946,  25950],
            [ 40522,  40669],
            [ 54509,  54590],
            [ 57887,  57892],
            [ 58918,  58925],
            [ 59572,  59580],
            [ 66366,  66394],
            [ 66569,  66576],
            [ 79730,  79765],
            [ 80228,  80256],
            [ 94908,  95482],
            [ 95575,  95577],
            [ 9